In [ ]:
import pandas as pd
from plotly import graph_objs as go
import matplotlib.pyplot as plt
import cv2
import face_recognition
import os

# 1. Data exploration

In [ ]:
data = pd.read_json("./data/train_sample_videos/metadata.json").T
data

In [ ]:
# Count number of label is REAL and FAKE 
data.label.value_counts()

# Plot REAL vs fake in plot
labels = data.label.value_counts().index
values = data.label.value_counts().values
# Plot
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()


A baseline for this project can be to always predict FAKE as this is what occurs most often

# 2. Data preparation

Finding faces in videos...

In [ ]:
path = "./data/train_sample_videos/"

# Get all files in directory
files = os.listdir(path)

# Only keep .mp4 files
files = [f for f in files if f.endswith(".mp4")]

for video in files:
    vidcap = cv2.VideoCapture(f"{path}{video}")

    def find_faces(image):
        face_locations = face_recognition.face_locations(image)
        for face_location in face_locations:
            # Print the location of each face in this image
            top, right, bottom, left = face_location
            # Access the actual face itself:
            face_image = image[top:bottom, left:right]
            return face_image

    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        face_image = None
        if hasFrames:
            face_image = find_faces(image)
        return hasFrames, face_image

    sec = 0
    frameRate = 0.5 # it will capture image in each 0.5 second
    count=1
    success = getFrame(sec)

    while success:
        sec = sec + frameRate
        sec = round(sec, 2)
        success, face_image = getFrame(sec)
        # Write image to folder
        if face_image is not None:
            # Ensure folder is created
            if not os.path.exists(f"./data/processed/{video}"):
                os.makedirs(f"./data/processed/{video}")
            cv2.imwrite(f"./data/processed/{video}/{count}.jpg", face_image)
        count = count + 1

    vidcap.release()
    cv2.destroyAllWindows()